In [9]:
"""
This script will sum up utilities by month and indicate how much is to be paid by each roommate.
It should track outstanding balances too
""";

In [10]:
import pandas as pd
from openpyxl import load_workbook

import numpy as np

import os
import glob

import datetime

## General I/O

In [11]:
def get_dtime (string):
    """
    converts strings to datetime objects using datetime.strptime mostly just wrote this so I could .apply() it in pandas
    method chains
    
    takes (string) which is the full date of format mm/dd/yyyy (can't be 2 digit year)
    """
    return datetime.datetime.strptime(string, '%m/%d/%Y')

In [12]:
def open_hist (filepath):
    
    sheet_dict = pd.read_excel(filepath, sheet_name=None)
    
#     hist['open'] = hist['open'].apply(get_dtime)
#     hist['close'] = hist['close'].apply(get_dtime)
    
#     hist['date'] = hist['date'].apply(get_dtime)
    
#     hist['st close'] = hist['st close'].apply(get_dtime)
#     hist['st open'] = hist['st open'].apply(get_dtime)
    
    return sheet_dict

## Calculating new balances

In [44]:
def roomie0_bal (utils_history_tidy):
    """
    ARAZIN
    """
    
    #get all the utils charges that roomie1 hasn't paid his share yet
    zero_unpaid = utils_history_tidy.loc[utils_history_tidy['arazin paid'].astype(str) == '0', :]
    
    #when it is just arazin and I, so total is split evenly by 2
    just_2_roomies = sum(zero_unpaid.loc[zero_unpaid['alex paid'].astype(str) == 'na', 'amnt'])/2
    
    #when we have the full 3 roommates
    full_house = sum(zero_unpaid.loc[zero_unpaid['alex paid'].astype(str) != 'na', 'amnt'])/3
    
    return (just_2_roomies + full_house, zero_unpaid)

In [45]:
def roomie1_bal (utils_history_tidy):
    """
    ALEX
    """
    
    #get all the utils charges that roomie2 hasn't paid his share yet
    one_unpaid = utils_history_tidy.loc[utils_history_tidy['alex paid'].astype(str) == '0', :]
    
    #when it is just alex and I, so total is split evenly by 2
    just_2_roomies = sum(one_unpaid.loc[one_unpaid['arazin paid'].astype(str) == 'na', 'amnt'])/2
    
    #when we have the full 3 roommates
    full_house = sum(one_unpaid.loc[one_unpaid['arazin paid'].astype(str) != 'na', 'amnt'])/3
    
    return (just_2_roomies + full_house, one_unpaid)

In [46]:
def bal_and_dates (utils_history_tidy, who):
    
    if who == 'arazin':
        bal, dates = roomie0_bal(utils_history_tidy)
    
    elif who == 'alex':
        bal, dates = roomie1_bal(utils_history_tidy)
        
    else:
        raise NameError('This is not a valid roommate')
    
    latest_unpaid_periods = pd.DataFrame()
    
    for util in np.unique(dates['util']):
    
        idx = dates['util'] == util

        open_per_util = dates.loc[idx, 'open']

        ma = max(open_per_util)

        idx_of_max = dates['open'] == ma

        max_per = dates.loc[idx_of_max, ['open', 'close']]
        
        max_per = max_per.assign(util = util)
        
        latest_unpaid_periods = pd.concat([latest_unpaid_periods, max_per])
    
    #get the date you did the calculation for reference
    now = datetime.datetime.now()
    
    latest_unpaid_periods = latest_unpaid_periods.assign(date_calc = datetime.datetime(now.year, now.month, now.day), 
                                                         who = who,
                                                         balance = bal)
        
    return latest_unpaid_periods

In [34]:
def write_current_balances (curr_bals_for_all, utils_hist_file):
    
    #load this file as an openpyxl workbook
    book = load_workbook(utils_hist_file)
    
    #create pandas writer object with the openpyxl engine
    writer = pd.ExcelWriter(utils_hist_file, engine='openpyxl')
    
    #tell the writer object the pertinent information about the openpyxl workbook
    #so it can do operations (write) in the right places in the file
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    #because we told the excel writer object the workbook and names of the sheets
    #it can write to the correct sheet and maintain all the data without deleting worksheets etc
    curr_bals_for_all.to_excel(writer, "balances", index=False)
    
    #save it
    writer.save()
    
    return None

In [35]:
def update_balances (file, roommates):
    #get the file and separate the sheets
    
    utils_dict = open_hist(file)

    utils_hist = utils_dict['utils']
    balances = utils_dict['balances']
    
    #get the new balances due
    all_new_bals = pd.DataFrame()
    for roomie in roommates:

        new_balance = bal_and_dates(utils_hist, roomie)

        all_new_bals = pd.concat([all_new_bals, new_balance], axis='index', sort=False)

    balances = pd.concat([balances, all_new_bals], axis=0, sort=False)
    
    
    #write it back
    
    #this does not currently check if I'm duplicating a balance
    #for now this is ok because balances calculated on different days still differ in their
    #'date calc' column and this is a good record of nonpayment and nonincrease over a period of time
    #also, the payment update function should erase all identical, and previous nonidentical balance amount entries
    write_current_balances(balances, file)
    
    print("The new balances have been calculated for {} and {} and are recorded in the 'balances' sheet of the tidy utils xlsx"
         .format(roommates[0], roommates[1]))
    
    return balances

In [36]:
def show_simple_bal (balances_df):
    
    simple = {}
    
    for person in np.unique(balances_df['who']):
        
        p_idx = balances_df['who'] == person
        
        max_calc_date_idx = balances_df.loc[:, 'date_calc'] == max(balances_df.loc[p_idx, 'date_calc'])
        
        amount = max(balances_df.loc[p_idx & max_calc_date_idx, 'balance'])
        
        simple.update({person : amount})
        
    for key in simple.keys():
        print("{} owes {}".format(key, simple[key]))
        
    return simple

## Deal with a payment

In [37]:
def update_util_hist_w_payment (amount_paid, who_paid, balances, utils_hist):
    
    #get all the balance sheet entries for the amount paid
    #(should be a single instance of each utility with open/close dates,
    #these dates are the latest unpaid period for that util at that balance amount)

    amnt_idx = balances['balance'] == amount_paid
    who_idx = balances['who'] == who_paid

    entries_for_amount = balances.loc[amnt_idx & who_idx, ['open', 'close', 'util']]
    
    
    #for each utility associated with the paid balance
    for util in np.unique(entries_for_amount['util']):

        #get the latest entry for a particular utility for the amount paid
        latest_unpaid_entry = entries_for_amount.loc[entries_for_amount['util'] == util, :]

        #find which entry, and all previous, in the utilities history is being paid in this amount
        #use the max() function in case the balance sheet has a history of multiple calculations of the same balance
        #Just in case the dates of the utility open/close have changed, I want to take the latest one of these and update
        #back from there
        open_idx = utils_hist['open'] <= max(latest_unpaid_entry['open'].values)
        close_idx = utils_hist['close'] <= max(latest_unpaid_entry['close'].values) #probably redundant
        util_idx = utils_hist['util'] == util

                  #don't update to 1 the places where the roommate didn't live there ('na')
        not_na_idx = (utils_hist[who_paid + ' paid'].astype(str) != 'na')


        #update this and all previous entries for this utility to 'paid' for this roommate
        utils_hist.loc[open_idx & close_idx & util_idx & not_na_idx, who_paid + ' paid'] = 1
        
    return utils_hist

In [38]:
def update_balances_w_payment (amount_paid, who_paid, balances):
    
    #find all the places in the balance sheet that are associated with the balance being paid
    amnt_idx = balances['balance'] == amount_paid
    who_idx = balances['who'] == who_paid
    
    #I don't want to remove just the balance entries with this value, because previous entries
    #that should get erased by this payment may have amounts less than this.
    #so let's get the latest balances calculation date for this amount and delete everything before this
    #this assumes there's never a case in which balances before the latest balance calc date won't be erased.
    latest_bal_calc_date = max(balances.loc[amnt_idx & who_idx, 'date_calc'])
    
    date_idx = balances['date_calc'] <= latest_bal_calc_date
    
    #the idxs above will allow me to access all the balance rows that correspond with this payment
    #but I want to get all the ones that ARENT this payment, and thereby drop those that are, so reverse it
    oppo = [not x for x in (amnt_idx & who_idx & date_idx)]
    
    balances = balances.loc[oppo]
    
    return balances

In [39]:
def write_hist_balances_w_payment (updated_utils_hist_df, updated_balances_df, utils_hist_file):
    
    #load this file as an openpyxl workbook
    book = load_workbook(utils_hist_file)
    
    #since we're overwriting this worksheet, let's get rid of it.
    #if you don't, it will only overwrite some of the data or none at all.
    #don't do this with the utilities hist because the length of that one doesn't change in this fxn
    book.remove(book['balances'])
    
    #create pandas writer object with the openpyxl engine
    writer = pd.ExcelWriter(utils_hist_file, engine='openpyxl')
    
    #tell the writer object the pertinent information about the openpyxl workbook
    #so it can do operations (write) in the right places in the file
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    #because we told the excel writer object the workbook and names of the sheets
    #it can write to the correct sheet and maintain all the data without deleting worksheets etc
    updated_utils_hist_df.to_excel(writer, "utils", index=False)
    updated_balances_df.to_excel(writer, "balances", index=False)
    
    #save it
    writer.save()
    
    return None

In [40]:
def process_payment (amount_paid, who_paid, utils_xlsx_filepath):
    
    #get the file you want to update with a payment
    utils_dict = open_hist(utils_xlsx_filepath)

    utils_hist = utils_dict['utils']
    balances = utils_dict['balances']
    
    #update the "paid" column for the person paying in the long utilities history sheet
    new_utils_hist = update_util_hist_w_payment (amount_paid, who_paid, balances, utils_hist)
    
    #delete the balances that exist for this amount for this person
    new_balances = update_balances_w_payment (amount_paid, who_paid, balances)
    
    #write it all to the file
    write_hist_balances_w_payment (new_utils_hist, new_balances, utils_xlsx_filepath)
    
    print('The payment in amount ${} from {} was processed. The utilities history reflects this payment and associated balances have been removed.'
         .format(amount_paid, who_paid))
    
    return new_utils_hist, new_balances

## Run below to calculate new balances

In [47]:
file = 'C:/Users/geeze/Documents/finances/utils/utilities_tidy.xlsx'

roommates = ['alex', 'arazin']

In [48]:
balances = update_balances(file, roommates)

The new balances have been calculated for alex and arazin and are recorded in the 'balances' sheet of the tidy utils xlsx


In [49]:
show_simple_bal(balances)

alex owes 81.05666666666667
arazin owes 237.23000000000002


{'alex': 81.05666666666667, 'arazin': 237.23000000000002}

## Run below when taking payments

In [62]:
file = 'C:/Users/geeze/Documents/finances/utils/utilities_tidy.xlsx'

#how much was paid
amount_paid = 155.84
#who paid you
roomie = 'arazin'

In [63]:
new_utils_hist, new_balances = process_payment (amount_paid, roomie, file)

The payment in amount $155.84 from arazin was processed. The utilities history reflects this payment and associated balances have been removed.
